# Random variables

The water monitoring project at the Montreux Jazz has been going on since 2016. The data has been collected and treated by a variety of people since then. 

__Objective:__ Standardize the nomenclature from the different sampling years. Provide a model for storing and collecting data in the future.

__Purpose:__ Define the probability that a survey will exceed a threshold value within the period of the year defined by the survey results.

## Definitions

* colony: a circular discoloration of the media within a defined size and color range
* colony-count: the number of discolorations of the same hue for a media type
* media/medium: the environment that the water samples are placed in
* color: the observed color of the colony
* label: the assumed category of the color:
  * Bioindicator
  * Coliform
  * Other
* coef:  the correction factor applied, to allow reporting of colony counts per 100ml of the original water sample.
 
The purpose of the sampling is to identify colonies that appear in the media and classify them as one of the possible labels. The label of interest is _Bioindicators_, this represents the bacteria that are issue from the organism of interest. The organism in this case is people, the _Bioindicator_ is issue from fecal contaminants.

## Methods

The process requires collaborating with the data-manager(s) from the different project years and ensuring that the data from each year can be combined and interpreted together. The data for this collaboration is stored in the _componentdata_ folder.

The relationship of previous label <---> new label is stored in a dictionary or an array for the different possibilities of medium, color, label and coefficient. The new labels are applied to a data-frame.

The finsihed data (the result of the collaboration) is stored in the _end_ folder

## Sample data

The sample data is an example of the desired output per year. This includes the following parameters:

1. colony-count
2. label
3. location
4. coeficient*count
5. week number
6. day of year
7. is-jazz: boolean
8. rain fall in millimeters

In [1]:
import pandas as pd
import datetime as dt
import numpy as np

project = "Hackuarium do it together water quality sampling"
site_markers = {"SVT":"o", "VNX":"D", "MRD":"X"}
species_colors = { "Bioindicator":"dodgerblue", "Coliform":"magenta"}
marker_colors = {"SVT":"black", "VNX":"green", "MRD":"goldenrod"}
sites = ["SVT", "VNX", "MRD"]

## Survey data

The format of the survey data after collaboration

In [2]:
stddf = pd.read_csv("data/end/survey_data_2020_2023.csv")
stddf['date'] = pd.to_datetime(stddf["date"])

stddf.head()

,date,location,sample,date_sample,event,before event,after event,medium,label,count,coef,week,doy,year,color,image
0,2023-06-12,VNX,VNX1,"('12/6/2023', 'VNX1')",False,True,False,ECC-A,Bioindicator,0.0,100,24,163,2023,Dark Blue,20230614_205159.jpg
1,2023-06-12,VNX,VNX1,"('12/6/2023', 'VNX1')",False,True,False,ECC-A,Other,0.0,100,24,163,2023,Turquoise,20230614_205159.jpg
2,2023-06-12,VNX,VNX1,"('12/6/2023', 'VNX1')",False,True,False,ECC-A,Coliform,11.0,100,24,163,2023,Pink,20230614_205159.jpg
3,2023-06-12,VNX,VNX2,"('12/6/2023', 'VNX2')",False,True,False,ECC-A,Bioindicator,1.0,100,24,163,2023,Dark Blue,20230614_205229.jpg
4,2023-06-12,VNX,VNX2,"('12/6/2023', 'VNX2')",False,True,False,ECC-A,Other,1.0,100,24,163,2023,Turquoise,20230614_205229.jpg


### Current data to process

There are two years left to process

* 2016
* 2017

```{error}
The data from 2016 does not have 48hour counts
```

In [3]:
new_df16 = pd.read_csv("data/componentdata/2016_Data.csv")

colors_2016 = [
    'P1_24h_big_blue', 'P1_24h_med_blue',
    'P1_24h_other', 'P1_24h_pink', 'P1_24h_turq', 'P1_qty_sample',
    'P2_24h_big_blue', 'P2_24h_med_blue', 'P2_24h_other', 'P2_24h_pink',
    'P2_24h_turq', 'P3_24h_big_blue', 'P3_24h_med_blue', 'P3_24h_other',
    'P3_24h_pink', 'P3_24h_turq']
colors_slug = {x:f'{x[1:3]}{x[7:]}' for x in colors_2016}

df16 = new_df16.rename(columns=colors_slug)
df16.drop("Unnamed: 0", axis=1, inplace=True)
df16 = pd.melt(df16, value_vars=colors_slug.values(), id_vars=["Date", "Location"])
df16["plate"] = df16.variable.apply(lambda x: x[0])
df16["sample"] = df16.Location + df16.plate
df16["colour"] = df16.variable.apply(lambda x: x[2:])
df16.drop("variable", inplace=True, axis=1)
df16.rename(columns={"value":"count", "Date":"date", "Location":"location"}, inplace=True)
df16["date"] = pd.to_datetime(df16["date"])
df16["image"] = None
df16["coef"] = 100

In [4]:
new_df16.columns

Index(['Unnamed: 0', 'Date', 'Location', 'P1_24h_big_blue', 'P1_24h_med_blue',
       'P1_24h_other', 'P1_24h_pink', 'P1_24h_turq', 'P1_qty_sample',
       'P2_24h_big_blue', 'P2_24h_med_blue', 'P2_24h_other', 'P2_24h_pink',
       'P2_24h_turq', 'P3_24h_big_blue', 'P3_24h_med_blue', 'P3_24h_other',
       'P3_24h_pink', 'P3_24h_turq'],
      dtype='object')

In [5]:
df16.head()

,date,location,count,plate,sample,colour,image,coef
0,2016-07-05,MRD,0.0,1,MRD1,big_blue,None,100
1,2016-07-12,MRD,22.0,1,MRD1,big_blue,None,100
2,2016-07-19,MRD,8.0,1,MRD1,big_blue,None,100
3,2016-06-21,MRD,2.0,1,MRD1,big_blue,None,100
4,2016-06-28,MRD,0.0,1,MRD1,big_blue,None,100


The data from 2017 will require quite a bit of formatting:

In [6]:
new_df17 = pd.read_csv("data/componentdata/2017_Data.csv")
new_df17.columns

Index(['Date', 'Location', 'medium', 'Samples', 'Sampling_Notes', 'Water_temp',
       'Plating_notes', 'Temp_incubation', 'P1_qty_sample',
       'Image_24h_fluo_plate_one', 'P1_fluo_halo_colonies', 'P1_fluo_other',
       'Plate_one_24h_image', 'P1_24h_big_blue', 'P1_24h_med_blue',
       'P1_24h_green', 'P1_24h_turq', 'P1_24h_pink', 'P1_24h_other',
       'Comments_p1_24h', 'Plate_one_48h_image', 'P1_48h_big_blue',
       'P1_48h_med_blue', 'P1_48h_green', 'P1_48h_turq', 'P1_48h_pink',
       'P1_48h_other', 'Comments_p1_48h', 'P2_qty_sample',
       'Image_24h_fluo_plate_two', 'P2_fluo_halo_colonies', 'P2_fluo_other',
       'Plate_two_24h_image', 'P2_24h_big_blue', 'P2_24h_med_blue',
       'P2_24h_green', 'P2_24h_turq', 'P2_24h_pink', 'P2_24h_other',
       'Comments_p2_24h', 'Plate_two_48h_image', 'P2_48h_big_blue',
       'P2_48h_med_blue', 'P2_48h_green', 'P2_48h_turq', 'P2_48h_pink',
       'P2_48h_other', 'Comments_p2_48h', 'P3_qty_sample',
       'Image_24h_fluo_plate_three

### Applying labels

The colors that were used for the observations can be placed into three broad categories. 

1. Bioindicator
2. Coliforms
3. Other

The microbiologist determines the correct label for the recorded color based on the specifics of the media/medium used to grow the culture.

The colors appropriate to each label are stored in an array. The color for each record is tested for membership in one of the arrays. If it is in one of the arrays, the name of that array is returned. If the color is not in any array the original value is returned. The result is added to the data-frame.

```python
bioindicators = ["Dark Blue", "Blue", "Turquoise fast", "metallic_green", "green_met", "fluo_halo", "big_blue"]
coliforms = ["Pink", "pink", "purple", "med_blue"]
other = ["Turquoise", "Turquoise slow", "other", "mauve", "fluo_other", "green"]

def translate_colors(x, bioindicators, coliforms, other):
    if x in bioindicators:
        return "Bioindicator"
    elif x in coliforms:
        return "Coliform"
    elif x in other:
        return "Other"
    else:
        return x

stddf ["label"] = stddf .color.apply(lambda x: translate_colors(x, bioindicators, coliforms, other))
```

We do the same for the media/medium except we use a dictionary to store that information

```python
media_names =  {
    "ECC-A Card":"ECC-A",
    "new ECCA":"ECC-A",
    "E-coli side": "E coli",
    "Double side E coli": "E coli",
    "ECC-side":"ECC",
    "Double side ECC":"ECC",
    "selective":"Levine",
    "media":"EasyGel",
    "plus uv":"EasyGelPlus",
    "UVplus":"EasyGelPlus",
    "non-restrictive":"LB",
    "levine": "Levine",
    "easy_gel":"EasyGel",
    "unil_kitchen":"LB",
    "micrology_card": "ECC"
}

def translate_media(x, media_names):
    if x in media_names.keys():
        return media_names[x]
    else:
        return x


stddf ["medium"] = stddf .media.apply(lambda x: translate_media(x, media_names))

```



### Labeling the date range of interest

Voici les dates de Jazz pour toutes les années de prélèvement :

* 2016:  2016-07-01 - 2016-07-16
* 2017: 2017-06-30 - 2017-06-15
* 2020: 2020-07-03 - 2020-07-18
* 2022: 2022-07-01 - 2022-07-16
* 2023: 2023-06-30 - 2023-07-15

__before event:__ samples before the begining of the event of interest

__after event:__ samples after the end of the event

In [7]:
# mask the date ranges
import datetime as dt

def make_date_object(x):
    return dt.datetime.strptime(x, "%Y-%m-%d")

# # stddf["before event"] = False
# # stddf["after event"] = False
# df["event"] = False
# event_mask = (df['date'] >= pd.Timestamp("2020-07-01")) & (df['date'] > pd.Timestamp("2020-07-16"))
# df.loc[event_mask, "event"] = True

# df["before event"] = False
# df["after event"] = False

# df.loc[ (df['date'] < pd.Timestamp("2020-07-01")), "before event"] = True
# df.loc[ (df['date'] > pd.Timestamp("2020-07-16")), "after event"] = True

# stddf["year"] = stddf["year"].astype("str")
# stddf.rename(columns={"isjazz": "event"}, inplace=True)

# stddf.loc[(stddf["year"] == "2022") & (stddf['date'] < pd.Timestamp("2022-07-01")), "before event"] = True
# stddf.loc[(stddf["year"] == "2023") & (stddf['date'] < pd.Timestamp("2023-06-30")), "before event"] = True
# stddf.loc[(stddf["year"] == "2022") & (stddf['date'] > pd.Timestamp("2022-07-16")), "after event"] = True
# stddf.loc[(stddf["year"] == "2023") & (stddf['date'] > pd.Timestamp("2023-07-15")), "after event"] = True
stddf.head()

,date,location,sample,date_sample,event,before event,after event,medium,label,count,coef,week,doy,year,color,image
0,2023-06-12,VNX,VNX1,"('12/6/2023', 'VNX1')",False,True,False,ECC-A,Bioindicator,0.0,100,24,163,2023,Dark Blue,20230614_205159.jpg
1,2023-06-12,VNX,VNX1,"('12/6/2023', 'VNX1')",False,True,False,ECC-A,Other,0.0,100,24,163,2023,Turquoise,20230614_205159.jpg
2,2023-06-12,VNX,VNX1,"('12/6/2023', 'VNX1')",False,True,False,ECC-A,Coliform,11.0,100,24,163,2023,Pink,20230614_205159.jpg
3,2023-06-12,VNX,VNX2,"('12/6/2023', 'VNX2')",False,True,False,ECC-A,Bioindicator,1.0,100,24,163,2023,Dark Blue,20230614_205229.jpg
4,2023-06-12,VNX,VNX2,"('12/6/2023', 'VNX2')",False,True,False,ECC-A,Other,1.0,100,24,163,2023,Turquoise,20230614_205229.jpg


In [8]:
df16["before event"] = False
df16["after event"] = False
df16["event"] = False
event_mask = (df16['date'] >= pd.Timestamp("2016-07-01")) & (df16['date'] <= pd.Timestamp("2016-07-16"))
df16.loc[event_mask, "event"] = True

df16.loc[ (df16['date'] < pd.Timestamp("2016-07-01")), "before event"] = True
df16.loc[ (df16['date'] > pd.Timestamp("2016-07-16")), "after event"] = True
df16.head()

,date,location,count,plate,sample,colour,image,coef,before event,after event,event
0,2016-07-05,MRD,0.0,1,MRD1,big_blue,None,100,False,False,True
1,2016-07-12,MRD,22.0,1,MRD1,big_blue,None,100,False,False,True
2,2016-07-19,MRD,8.0,1,MRD1,big_blue,None,100,False,True,False
3,2016-06-21,MRD,2.0,1,MRD1,big_blue,None,100,True,False,False
4,2016-06-28,MRD,0.0,1,MRD1,big_blue,None,100,True,False,False


In [14]:
df16["d"] = df16["date"].dt.strftime('%Y-%m-%d')
df16["date_sample"] = list(zip(df16["d"], df16["sample"]))

In [15]:
# translate colors
def translate_colors(x, bioindicators, coliforms, other):
    if x in bioindicators:
        return "Bioindicator"
    elif x in coliforms:
        return "Coliform"
    elif x in other:
        return "Other"
    else:
        return x

bioindicators = ["Dark Blue", "Blue", "Turquoise fast", "metallic_green", "green_met", "fluo_halo", "big_blue"]
coliforms = ["Pink", "pink", "purple", "med_blue"]
other = ["Turquoise", "Turquoise slow", "other", "mauve", "fluo_other", "green", "turq", 'sample']

# df["label"] = df.color.apply(lambda x: translate_colors(x, bioindicators, coliforms, other))

def translate_media(x, media_names):
    if x in media_names.keys():
        return media_names[x]
    else:
        return x

media_names =  {
"ECC-A Card":"ECC-A",
"new ECCA":"ECC-A",
"E-coli side": "E coli",
"Double side E coli": "E coli",
"ECC-side":"ECC",
"Double side ECC":"ECC",
"selective":"Levine",
"media":"EasyGel",
"plus uv":"EasyGelPlus",
"UVplus":"EasyGelPlus",
"non-restrictive":"LB",
"levine": "Levine",
"easy_gel":"EasyGel",
"unil_kitchen":"LB",
"micrology_card": "ECC"
}

# df["medium"] = df.media.apply(lambda x: translate_media(x, media_names))

In [19]:
df16["label"] = df16.colour.apply(lambda x: translate_colors(x, bioindicators, coliforms, other))
df16["medium"] = "EasyGel"
df16["doy"] = df16["date"].dt.dayofyear
df16["week"] = df16["date"].dt.isocalendar().week
df16["year"] = df16["date"].dt.year

In [20]:
df16.date_sample.unique()

array([('2016-07-05', 'MRD1'), ('2016-07-12', 'MRD1'),
       ('2016-07-19', 'MRD1'), ('2016-06-21', 'MRD1'),
       ('2016-06-28', 'MRD1'), ('2016-07-26', 'MRD1'),
       ('2016-08-02', 'MRD1'), ('2016-08-09', 'MRD1'),
       ('2016-07-05', 'SVT1'), ('2016-07-12', 'SVT1'),
       ('2016-07-19', 'SVT1'), ('2016-06-21', 'SVT1'),
       ('2016-06-28', 'SVT1'), ('2016-07-26', 'SVT1'),
       ('2016-08-02', 'SVT1'), ('2016-08-09', 'SVT1'),
       ('2016-07-05', 'VNX1'), ('2016-07-12', 'VNX1'),
       ('2016-07-19', 'VNX1'), ('2016-06-21', 'VNX1'),
       ('2016-06-28', 'VNX1'), ('2016-07-26', 'VNX1'),
       ('2016-08-09', 'VNX1'), ('2016-08-02', 'VNX1'),
       ('2016-07-05', 'MRD2'), ('2016-07-12', 'MRD2'),
       ('2016-07-19', 'MRD2'), ('2016-06-21', 'MRD2'),
       ('2016-06-28', 'MRD2'), ('2016-07-26', 'MRD2'),
       ('2016-08-02', 'MRD2'), ('2016-08-09', 'MRD2'),
       ('2016-07-05', 'SVT2'), ('2016-07-12', 'SVT2'),
       ('2016-07-19', 'SVT2'), ('2016-06-21', 'SVT2'),
       ('2

In [21]:
df16.to_csv("data/end/survey_data_2016.csv")

# Rain fall

Expected format of rain data

In [17]:
sample_data = pd.read_csv("data/end/rain_data_2016.csv")
sample_data.head()

,date,mm
0,2016-06-21,4.0
1,2016-06-22,0.6
2,2016-06-23,0.9
3,2016-06-24,13.1
4,2016-06-25,9.8


In [18]:
merge_columns = [
    "date",
    "location",
    "sample",
    "date_sample",
    "event",
    "before event",
    "after event",
    "medium",
    "label",
    "count",
    "coef",
    "week",
    "doy",
    "year",
    "color",
    "image",
    
]